In [3]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [9]:
# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255

In [10]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [13]:
# Create a model and add layers
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [14]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [15]:
# Print a summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

In [16]:
# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path('model_structure.json')
f.write_text(model_structure)

# Save neural network's trained weights
model.sample_weights('model_weights.h5')

Epoch 1/30
782/782 [==============================] - 106s 133ms/step - loss: 1.7889 - accuracy: 0.3353 - val_loss: 1.1956 - val_accuracy: 0.5702
Epoch 2/30
782/782 [==============================] - 104s 133ms/step - loss: 1.2076 - accuracy: 0.5710 - val_loss: 0.9845 - val_accuracy: 0.6522
Epoch 3/30
782/782 [==============================] - 101s 130ms/step - loss: 1.0285 - accuracy: 0.6355 - val_loss: 0.8792 - val_accuracy: 0.6933
Epoch 4/30
782/782 [==============================] - 101s 129ms/step - loss: 0.9041 - accuracy: 0.6829 - val_loss: 0.8223 - val_accuracy: 0.7128
Epoch 5/30
782/782 [==============================] - 102s 130ms/step - loss: 0.8367 - accuracy: 0.7041 - val_loss: 0.7734 - val_accuracy: 0.7326
Epoch 6/30
782/782 [==============================] - 104s 133ms/step - loss: 0.7755 - accuracy: 0.7286 - val_loss: 0.7467 - val_accuracy: 0.7406
Epoch 7/30
782/782 [==============================] - 108s 138ms/step - loss: 0.7314 - accuracy: 0.7429 - val_loss: 0.7150 -

AttributeError: 'Sequential' object has no attribute 'sample_weights'

In [39]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np

In [34]:
# These are the CIFAR10 class labels from the training data (in order from 0 to 9)
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

In [19]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

In [26]:
# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

In [27]:
# Re-load the model's trained weights
model.load_weights("model_weights.h5")

In [36]:
# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("frog.png", target_size=(32, 32))

In [37]:
# Convert the image to a numpy array
image_to_test = image.img_to_array(img)

In [40]:
# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)

In [41]:
# Make a prediction using the model
results = model.predict(list_of_images)

In [42]:
# Since we are only testing one image, we only need to check the first result
single_result = results[0]

In [43]:
# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

In [44]:
# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

In [45]:
class_label

'Frog'

In [50]:
# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("bird.png", target_size=(32, 32))

# Convert the image to a numpy array
image_to_test = image.img_to_array(img)

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)

# Make a prediction using the model
results = model.predict(list_of_images)

# Since we are only testing one image, we only need to check the first result
single_result = results[0]

# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

class_label

'Frog'